In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import tensorflow.keras
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import TensorBoard
from time import time
import tensorflow as tf
import os

In [ ]:
%load_ext tensorboard

In [ ]:
#early stopping
 # This callback will stop the training when there is no improvement in
 # the loss for three consecutive epochs.
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3) #>val_loss

#Create a checkpoint
logdir = os.path.join("/content/drive/MyDrive/DEEP_LOG/logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
checkpoint_filepath = logdir + '/checkpoint/'
print(checkpoint_filepath)


model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)



#scheduler Function
def lr_schedule(epoch):
  """
  Returns a custom learning rate that decreases as epochs progress.
  """
  learning_rate = 0.2
  if epoch > 10:
    learning_rate = 0.02
  if epoch > 20:
    learning_rate = 0.01
  if epoch > 50:
    learning_rate = 0.005
  
  tf.summary.scalar('learning rate', data=learning_rate, step=epoch)
  return learning_rate

In [ ]:
#initializing LearningRateScheduler
callback = tf.keras.callbacks.LearningRateScheduler(lr_schedule)

import datetime
#to see learning rate
logdir = os.path.join("/content/drive/MyDrive/DEEP_LOG/logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
file_writer = tf.summary.create_file_writer(logdir + "/metrics")
file_writer.set_as_default()

# Model configuration
img_width, img_height = 32, 32
batch_size = 250
no_epochs = 40
no_classes = 10
validation_split = 0.2
verbosity = 1

In [ ]:

# Load CIFAR10 dataset
(input_train, target_train), (input_test, target_test) = cifar10.load_data()

# Visualize CIFAR10 dataset
import matplotlib.pyplot as plt
classes = {
  0: 'airplane',
  1: 'automobile',
  2: 'bird',
  3: 'cat',
  4: 'deer',
  5: 'dog',
  6: 'frog',
  7: 'horse',
  8: 'ship',
  9: 'truck'
}
fig, axes = plt.subplots(2,5, sharex=True)
axes[0,0].imshow(input_train[0])
axes[0,1].imshow(input_train[1])
axes[0,2].imshow(input_train[2])
axes[0,3].imshow(input_train[3])
axes[0,4].imshow(input_train[4])
axes[1,0].imshow(input_train[5])
axes[1,1].imshow(input_train[6])
axes[1,2].imshow(input_train[7])
axes[1,3].imshow(input_train[8])
axes[1,4].imshow(input_train[9])
axes[0,0].set_title(classes[target_train[0][0]])
axes[0,1].set_title(classes[target_train[1][0]])
axes[0,2].set_title(classes[target_train[2][0]])
axes[0,3].set_title(classes[target_train[3][0]])
axes[0,4].set_title(classes[target_train[4][0]])
axes[1,0].set_title(classes[target_train[5][0]])
axes[1,1].set_title(classes[target_train[6][0]])
axes[1,2].set_title(classes[target_train[7][0]])
axes[1,3].set_title(classes[target_train[8][0]])
axes[1,4].set_title(classes[target_train[9][0]])
axes[0,0].set_axis_off()
axes[0,1].set_axis_off()
axes[0,2].set_axis_off()
axes[0,3].set_axis_off()
axes[0,4].set_axis_off()
axes[1,0].set_axis_off()
axes[1,1].set_axis_off()
axes[1,2].set_axis_off()
axes[1,3].set_axis_off()
axes[1,4].set_axis_off()
plt.show()

In [ ]:
input_shape = (img_width, img_height, 3)

# Parse numbers as floats
input_train = input_train.astype('float32')
input_test = input_test.astype('float32')

# Convert them into black or white: [0, 1].
input_train = input_train / 255
input_test = input_test / 255

# Convert target vectors to categorical targets
target_train = tensorflow.keras.utils.to_categorical(target_train, no_classes)
target_test = tensorflow.keras.utils.to_categorical(target_test, no_classes)

In [ ]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(no_classes, activation='softmax'))

In [ ]:
def Train():
  model.compile(loss=tensorflow.keras.losses.categorical_crossentropy,
                optimizer=tensorflow.keras.optimizers.Adam(),
                metrics=['accuracy'])
  
  # Define Tensorboard as a Keras callback
  tensorboard = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

  keras_callbacks = [
  tensorboard , callback , model_checkpoint_callback , early_stopping
]

  model.fit(input_train, target_train,
            batch_size=batch_size,
            epochs=no_epochs,
            verbose=verbosity,
            validation_split=validation_split,

            callbacks=keras_callbacks)

  # Generate generalization metrics
  score = model.evaluate(input_test, target_test, verbose=0)
  print(f'Test loss: {score[0]} / Test accuracy: {score[1]}')


Train()

In [ ]:
%tensorboard --logdir "/content/drive/MyDrive/DEEP_LOG/logs"

In [ ]:
model.load_weights(checkpoint_filepath)